In [1]:
#import torch
import webdataset
import json
import pandas as pd
import io
from mcap.reader import McapReader
from mcap.reader import make_reader 
import cv2
import numpy as np
import tempfile
import os
import time
from IPython.display import display, Image, clear_output


In [2]:
SHARD_PATH = "/Users/pierrebelamri/Downloads/shard-000639.tar"

dataset = webdataset.WebDataset(SHARD_PATH)


/opt/miniconda3/envs/vjepa/lib/python3.12/site-packages/webdataset/compat.py:379: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
  warnings.warn("WebDataset(shardshuffle=...) is None; set explicitly to False or a number")


In [3]:
dataset_iterator = dataset.iterator()

In [4]:
all_keys = []

for sample in dataset:
    jsonl_bytes = sample['.jsonl']
    jsonl_data_string = jsonl_bytes.decode('utf-8')
    df = pd.read_json(io.StringIO(jsonl_data_string), lines=True)

    for row in df['keyboard'].apply(lambda x: x["keys"]):
        all_keys+=row

In [5]:
np.unique(all_keys)

array(['key.keyboard.a', 'key.keyboard.caps.lock', 'key.keyboard.d',
       'key.keyboard.e', 'key.keyboard.escape', 'key.keyboard.left.alt',
       'key.keyboard.left.shift', 'key.keyboard.s', 'key.keyboard.space',
       'key.keyboard.w', 'scancode.0'], dtype='<U23')

In [6]:
# hotbar 
[f'key.keyboard.{i}' for i in range(1,10)] + \
[f'key.keyboard.{x}' for x in ['w', 'a', 's', 'd', 'space']] + \
[f'key.keyboard.{x}' for x in ['q','f']] + \
[f'key.keyboard.{x}' for x in ['e']] + \
[f'key.keyboard.{x}' for x in ['key.keyboard.escape']] + \
[f'key.keyboard.{x}' for x in ['left.shift', 'right.shift', 'left.control', ]] 

['key.keyboard.1',
 'key.keyboard.2',
 'key.keyboard.3',
 'key.keyboard.4',
 'key.keyboard.5',
 'key.keyboard.6',
 'key.keyboard.7',
 'key.keyboard.8',
 'key.keyboard.9',
 'key.keyboard.w',
 'key.keyboard.a',
 'key.keyboard.s',
 'key.keyboard.d',
 'key.keyboard.space',
 'key.keyboard.q',
 'key.keyboard.f',
 'key.keyboard.e',
 'key.keyboard.key.keyboard.escape',
 'key.keyboard.left.shift',
 'key.keyboard.right.shift',
 'key.keyboard.left.control']

In [ ]:
x0=128
x1=x0+10

df['hotbar'][x0:x1]

In [ ]:
print(df.head())

In [ ]:
for keyboard in df['keyboard']:
    if 'key.keyboard.space' in keyboard['keys']:
        break

In [ ]:
keyboard

In [ ]:
print(df['keyboard'][133])
print(df['keyboard'][134])

In [ ]:
print(df['mouse'][133])
print(df['mouse'][134])

In [ ]:
df['hotbar'][133:135]


In [ ]:
df['mouse'][x0:x1]

In [ ]:
all_mouse_buttons =[]

for sample in dataset:
    jsonl_bytes = sample['.jsonl']
    jsonl_data_string = jsonl_bytes.decode('utf-8')
    df = pd.read_json(io.StringIO(jsonl_data_string), lines=True)

    for row in df['mouse'].apply(lambda x: x["buttons"]):
        all_mouse_buttons+=row

In [ ]:
pd.value_counts(all_mouse_buttons)

In [ ]:
keyboard_states = df['keyboard'].values[:-1]

# Create a (T-1, num_keys) zero array
#key_presses_np = np.zeros((len(keyboard_states), len(self.KEYBOARD_KEYS)), dtype=np.float32)



In [ ]:
keyboard_states

In [ ]:
df['yaw'].values.shape

In [ ]:
import pandas as pd
import io






df['keyboard'] = df.apply(modify_keyboard_on_change, axis=1)


df = df.drop(columns=['next_hotbar', 'hotbar_changed'])


In [ ]:
print(f"Row 133 keyboard: {df.loc[133]['keyboard']}")
print(f"Row 135 keyboard: {df.loc[134]['keyboard']}")

In [ ]:
df['hotbar']

In [ ]:
sample = next(iter(dataset))

In [ ]:
print(f"Found sample with keys: {list(sample.keys())}")
print("The key '__key__' is the base name of the files.")
print(f"Base name: {sample['__key__']}\n")


In [ ]:
video_bytes = sample['.mp4']
print(f"Type: {type(video_bytes)}")
print(f"Size: {len(video_bytes) / (1024*1024):.2f} MB")
print("This is the raw byte content of the video file.")
print("You would later feed this to a video-loading library (like decord or opencv).\n")



In [ ]:
jsonl_bytes = sample['.jsonl']
jsonl_data_string = jsonl_bytes.decode('utf-8')
df = pd.read_json(io.StringIO(jsonl_data_string), lines=True)

print(f"Loaded .jsonl into a pandas DataFrame with {len(df)} rows (actions).")
print("First 5 actions:")
print(df.head())
print("\nDataFrame Info:")
df.info()


In [ ]:
np.unique(all_keys)

In [ ]:
mcap_bytes = sample['.mcap']
print(f"Type: {type(mcap_bytes)}")
print(f"Size: {len(mcap_bytes) / 1024:.2f} KB\n")


mcap_file_like = io.BytesIO(mcap_bytes)
reader = make_reader(mcap_file_like)

print("Iterating through first 3 messages in the .mcap file...")


count = 0
for schema, channel, message in reader.iter_messages():
    #if count >= 30:
    #    break
    
    if channel.topic == 'keyboard':

    # Now we use the unpacked variables correctly:
        print(f"  [{count}] Topic: {channel.topic}") 
        print(f"      Schema: {schema.name}")
        print(f"      Log Time: {message.log_time}")
    # message.data is the parsed dictionary
        print(f"      Data: {message.data}") 
    count += 1

if count == 0:
    print("No messages found in the .mcap file.")




In [ ]:

video_bytes = sample['.mp4']

# --- 2. Create and write to a temporary file ---
# We use tempfile to safely create a file
# 'delete=False' means we have to manually delete it later
with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as temp_file:
    temp_file.write(video_bytes)
    temp_video_path = temp_file.name
    print(f"Video bytes saved to temporary file: {temp_video_path}")

# --- 3. Read the video with OpenCV ---
cap = cv2.VideoCapture(temp_video_path)

displaying = False

if not cap.isOpened():
    print("Error: Could not open temporary video file.")
else:
    print("\nDisplaying all video frames (simulated video):")
    frame_count = 0
    try:
        while cap.isOpened() and frame_count < 1e10:
            ret, frame = cap.read()

            if not ret:
                print("End of video.")
                break
            
            if displaying :
                # b. Encode the frame as PNG bytes to display
                _, png_data = cv2.imencode('.png', frame)

                # c. Clear previous frame and display the new one
                clear_output(wait=True)
                display(Image(data=png_data.tobytes()))

                # d. Wait a bit to simulate video playback
                time.sleep(0.05) 

            frame_count += 1

    except KeyboardInterrupt:
        print("Video playback stopped by user.")
    finally:
        # --- 5. Clean up ---
        cap.release()
        os.unlink(temp_video_path)
        print(f"Cleaned up temporary file: {temp_video_path}")



In [ ]:
df

In [ ]:
png_data

In [ ]:
frame_count

In [ ]:
count

# Dataset implementation

In [20]:
import sys
sys.path.append('vjepa2')
import logging
import joblib
import torch
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from vjepa2.app.vjepa_minecraft.vpt_dataset import init_vpt_dataloader



In [21]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

DATA_PATH = SHARD_PATH

BATCH_SIZE = 1
FRAMES_PER_CLIP = 16
FPS = 5
FRAMESKIP = 2  # This is the `tubelet_size`
CROP_SIZE = 256
NUM_WORKERS = 0 # Set to 0 for initial debugging, then increase

# --- Initialize Loader ---
try:
    data_loader, _ = init_vpt_dataloader(
        data_path=DATA_PATH,
        batch_size=BATCH_SIZE,
        frames_per_clip=FRAMES_PER_CLIP,
        fps=FPS,
        crop_size=CROP_SIZE,
        rank=0,
        world_size=1,
        frameskip=FRAMESKIP,
        num_workers=NUM_WORKERS,
    )

    logger.info("Fetching one batch...")
    batch = next(iter(data_loader))

    video_tensor = batch['video_frames']
    actions_tensor = batch['actions']
    states_tensor = batch['states']

    logger.info(f"Batch loaded successfully!")

    # (B, T, C, H, W)
    logger.info(f"  Video tensor shape: {video_tensor.shape}") 

    # (B, T_actions, D_actions)
    # T_actions = (FRAMES_PER_CLIP / FRAMESKIP) - 1 = (16/2) - 1 = 7
    logger.info(f"  Actions tensor shape: {actions_tensor.shape}")

    # (B, T_states, D_states)
    # T_states = FRAMES_PER_CLIP / FRAMESKIP = 16/2 = 8
    logger.info(f"  States tensor shape: {states_tensor.shape}")

except FileNotFoundError as e:
    logger.error(f"Error: {e}")
    logger.error("Please update the DATA_PATH variable in the __main__ block.")
except KeyError as e:
     logger.error(f"Error: Missing data key {e}.")
     logger.error("This could be a column in the DataFrame (e.g., 'hotbar')")
     logger.error("OR a key in a dict (e.g., 'dx' in the 'mouse' column).")
     logger.error("Please verify all keys in vpt_states_to_diffs().")
except Exception as e:
    logger.error(f"An unexpected error occurred: {e}")
    import traceback
    traceback.print_exc()

[INFO    ][2025-12-24 19:58:06][vjepa2.app.vjepa_minecraft.vpt_dataset][init_vpt_dataloader      ] Initializing VPT WebDataset loader from: /Users/pierrebelamri/Downloads/shard-000639.tar
[INFO    ][2025-12-24 19:58:06][vjepa2.app.vjepa_minecraft.vpt_dataset][init_vpt_dataloader      ] Found 1 shards.
[INFO    ][2025-12-24 19:58:06][vjepa2.app.vjepa_minecraft.vpt_dataset][init_vpt_dataloader      ] VPT WebDataset data loader created
[INFO    ][2025-12-24 19:58:06][root                ][<module>                 ] Fetching one batch...


/opt/miniconda3/envs/vjepa/lib/python3.12/site-packages/webdataset/compat.py:379: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
  warnings.warn("WebDataset(shardshuffle=...) is None; set explicitly to False or a number")
/opt/miniconda3/envs/vjepa/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


[INFO    ][2025-12-24 19:58:10][root                ][<module>                 ] Batch loaded successfully!
[INFO    ][2025-12-24 19:58:10][root                ][<module>                 ]   Video tensor shape: torch.Size([1, 8, 3, 256, 256])
[INFO    ][2025-12-24 19:58:10][root                ][<module>                 ]   Actions tensor shape: torch.Size([1, 7, 38])
[INFO    ][2025-12-24 19:58:10][root                ][<module>                 ]   States tensor shape: torch.Size([1, 8, 2])


In [17]:
actions_tensor.shape

torch.Size([1, 7, 38])

In [3]:
actions_tensor

NameError: name 'actions_tensor' is not defined

In [18]:
states_tensor.shape

torch.Size([1, 8, 2])

In [2]:
len([f'key.keyboard.{i}' for i in range(1,10)] + \
    [f'key.keyboard.{x}' for x in ['w', 'a', 's', 'd', 'space']] + \
    [f'key.keyboard.{x}' for x in ['q','f']] + \
    [f'key.keyboard.{x}' for x in ['e']] + \
    [f'key.keyboard.{x}' for x in ['escape']] + \
    [f'key.keyboard.{x}' for x in ['left.shift', 'right.shift', 'left.control', ]] )\
+len([0, 1, 2])  \
#+9 

24

In [ ]:
for x in scaler_fit_loader:
    break

In [ ]:

    scaler = StandardScaler()
    
    logger.info("Starting scaler fitting loop...")
    
    with torch.no_grad():
        for batch in tqdm(scaler_fit_loader, desc="Fitting Scaler"):
            actions_batch_tensor = batch['actions']
            continuous_actions_tensor = actions_batch_tensor[:, :, :4]
            continuous_actions_flat = continuous_actions_tensor.reshape(-1, 4)
            continuous_actions_np = continuous_actions_flat.cpu().numpy()
            if len(continuous_actions_np) > 0:
                scaler.partial_fit(continuous_actions_np)

    joblib.dump(scaler, scaler_save_path)
    
    logger.info("=" * 30)
    logger.info(f"Scaler fitting complete and saved to: {scaler_save_path}")
    logger.info(f"  Mean: {scaler.mean_}")
    logger.info(f"  Scale (StdDev): {scaler.scale_}")
    logger.info("=" * 30)
    
    return scaler



In [24]:
import sys
sys.path.append('vjepa2')
import logging
import joblib
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from vjepa2.app.vjepa_minecraft.vpt_dataset import init_vpt_dataloader

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

SCALER_PATH = "vpt_action_scaler.pkl"

# OPTIMIZATION TIP: 
# Batch size 1 is very slow for statistics gathering. 
# Increase this to max out your GPU memory (e.g., 64, 128, 256).
FIT_BATCH_SIZE = 4 
# Increase workers to prefetch data while GPU computes
NUM_WORKERS = 0 

def fit_action_scaler_fast(data_path, scaler_save_path):
    logger.info(f"Initializing dataloader to fit scaler from: {data_path}")

    scaler_fit_loader, _ = init_vpt_dataloader(
        data_path=data_path,
        batch_size=FIT_BATCH_SIZE,
        action_scaler=None,
        frames_per_clip=64,
        fps=5,
        frameskip=2,
        crop_size=256,
        rank=0,
        world_size=1,
        num_workers=NUM_WORKERS,
        drop_last=False,
    )

    # Initialize accumulators on the device (GPU if available)
    # Assuming 4 continuous action dimensions based on your snippet
    device = torch.device("cpu")
    
    total_count = 0
    total_sum = torch.zeros(4, device=device, dtype=torch.float64) # Use float64 for precision
    total_sq_sum = torch.zeros(4, device=device, dtype=torch.float64)

    logger.info("Starting fast scaler fitting loop (Sufficient Statistics)...")
    
    with torch.no_grad():
        for batch in tqdm(scaler_fit_loader, desc="Collecting Stats"):
            # Extract and reshape
            actions_batch = batch['actions'].to(device)
            cont_actions = actions_batch[..., :4].reshape(-1, 4).double() # Cast to double for precision

            if cont_actions.shape[0] > 0:
                # Accumulate stats purely in PyTorch
                total_count += cont_actions.shape[0]
                total_sum += torch.sum(cont_actions, dim=0)
                total_sq_sum += torch.sum(cont_actions ** 2, dim=0)

    # --- Final Calculation ---
    # Calculate Mean: E[X]
    mean = total_sum / total_count
    
    # Calculate variance: E[X^2] - (E[X])^2
    variance = (total_sq_sum / total_count) - (mean ** 2)
    
    # handle precision issues
    variance = torch.clamp(variance, min=0.0)
    std_dev = torch.sqrt(variance)

    # Move to CPU/Numpy for Scikit-Learn
    mean_np = mean.cpu().numpy()
    std_np = std_dev.cpu().numpy()
    var_np = variance.cpu().numpy()

    # --- Manually Inject into StandardScaler ---
    # We create the object and force-feed the calculated attributes
    scaler = StandardScaler()
    scaler.mean_ = mean_np
    scaler.scale_ = std_np
    scaler.var_ = var_np
    scaler.n_samples_seen_ = total_count
    
    # Important: Set copy=True and with_mean/std=True to ensure it works correctly
    scaler.with_mean = True
    scaler.with_std = True
    scaler.copy = True

    # Save
    joblib.dump(scaler, scaler_save_path)
    
    logger.info("=" * 30)
    logger.info(f"Scaler fitting complete and saved to: {scaler_save_path}")
    logger.info(f"  Total Samples: {total_count}")
    logger.info(f"  Mean: {scaler.mean_}")
    logger.info(f"  Scale (StdDev): {scaler.scale_}")
    logger.info("=" * 30)
    
    return scaler

if __name__ == "__main__":
    try:
        fit_action_scaler_fast(DATA_PATH, SCALER_PATH)
        
        # Verify
        logger.info(f"Loading scaler from {SCALER_PATH} for verification...")
        loaded_scaler = joblib.load(SCALER_PATH)
        # Test a transform to ensure internal state is valid
        dummy_data = np.random.rand(5, 4)
        scaled_dummy = loaded_scaler.transform(dummy_data)
        logger.info("Verification transform successful.")
        
    except Exception as e:
        logger.error(f"An unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()

[INFO    ][2025-12-24 21:02:42][root                ][fit_action_scaler_fast   ] Initializing dataloader to fit scaler from: /Users/pierrebelamri/Downloads/shard-000639.tar
[INFO    ][2025-12-24 21:02:42][vjepa2.app.vjepa_minecraft.vpt_dataset][init_vpt_dataloader      ] Initializing VPT WebDataset loader from: /Users/pierrebelamri/Downloads/shard-000639.tar
[INFO    ][2025-12-24 21:02:42][vjepa2.app.vjepa_minecraft.vpt_dataset][init_vpt_dataloader      ] Found 1 shards.
[INFO    ][2025-12-24 21:02:42][vjepa2.app.vjepa_minecraft.vpt_dataset][init_vpt_dataloader      ] VPT WebDataset data loader created
[INFO    ][2025-12-24 21:02:42][root                ][fit_action_scaler_fast   ] Starting fast scaler fitting loop (Sufficient Statistics)...


  warnings.warn(warn_msg)


KeyboardInterrupt: 